In [1]:
!pip install -U 'git+https://github.com/facebookresearch/detectron2.git'

!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2' detectron2_repo
dist = distutils.core.run_setup("./detectron2_repo/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}

!pip install git+https://github.com/facebookresearch/detectron2@main#subdirectory=projects/DensePose

!pip install rembg
!pip install onnxruntime

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-q6bq197v
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-q6bq197v
  Resolved https://github.com/facebookresearch/detectron2.git to commit 525208d9d6b36c72500183470fd181848b822468
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.0 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=6730663 sha256=cbfda80f48a62f77b7248329fa81726bff397dcf2b9d091244e0b45733da2223
  Stored in directory: /tmp/pip-ephem-wheel-cache-4le3vy89/wheels/d3/6e/bd/1969578f1456a6be2

In [2]:
# app.py
import gradio as gr
import torch
from PIL import Image
import numpy as np
from diffusers import AutoencoderKL
from viton_pipeline import VitonPipeline

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load checkpoints and VAE
checkpoint_ddpm_path = '/content/drive/MyDrive/AIClothes/DDPM/Models/Diffusion/latent512_1500img_resnet_viton_adamW_schedulelr1e-05_vgg0.001_epoch_380.pth'
checkpoint_warping_path = '/content/drive/MyDrive/AIClothes/DDPM/Models/Warping/3000imgs_warping_lr5e-05_vgg0.1_tvlmabda_1epoch_30.pth'

checkpoint_ddpm = torch.load(checkpoint_ddpm_path, map_location=device)
checkpoint_warping = torch.load(checkpoint_warping_path, map_location=device)

vae = AutoencoderKL.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="vae"
).to(device)

# Initialize VitonPipeline
viton_pipeline = VitonPipeline((512, 512), checkpoint_ddpm, checkpoint_warping, vae)

def tensor_as_image(tensor, filename="output.png"):
    # Check tensor shape
    if tensor.shape != (1, 3, 512, 512):
        raise ValueError("Expected tensor shape [1, 3, 512, 512]")

    # Remove batch dimension
    tensor = tensor.squeeze(0)  # Shape: [3, 512, 512]

    # Convert tensor to PIL Image
    # Assuming tensor is on GPU, move to CPU and convert to numpy
    tensor = tensor.cpu().detach()

    # If tensor is in [0, 1], scale to [0, 255]
    tensor = ((tensor + 1)/2)*255

    # Convert to uint8
    tensor = tensor.clamp(0, 255).byte()

    # Permute dimensions from [C, H, W] to [H, W, C] for PIL
    tensor = tensor.permute(1, 2, 0)  # Shape: [512, 512, 3]

    # Convert to numpy array
    image_array = tensor.numpy()

    # Create PIL Image
    image = Image.fromarray(image_array)

    return image

def process_images(person_img, cloth_img):
    """
    Process the person and cloth images through the VitonPipeline.

    Args:
        person_img: PIL Image of the person
        cloth_img: PIL Image of the cloth

    Returns:
        PIL Image of the output
    """
    # Run the pipeline
    output = viton_pipeline(person_img, cloth_img)

    # Convert output tensor to PIL Image
    output_img = tensor_as_image(output)

    return output_img

# Define Gradio interface
iface = gr.Interface(
    fn=process_images,
    inputs=[
        gr.Image(type="pil", label="Person Image"),
        gr.Image(type="pil", label="Cloth Image")
    ],
    outputs=gr.Image(type="pil", label="Output Image"),
    title="VITON Virtual Try-On",
    description="Upload an image of a person and a cloth to see the virtual try-on result."
)

# Launch the interface
if __name__ == "__main__":
    iface.launch(debug=True)

SyntaxError: invalid syntax (viton_pipeline.py, line 10)